<div style="width: 100%; clear: both;">
    <div style="float: left; width: 50%;">
        <img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg" , align="left">
    </div>
</div>
<div style="float: right; width: 50%;">
    <p style="margin: 0; padding-top: 22px; text-align:right;">22.421 · Anàlisi de xarxes socials</p>
    <p style="margin: 0; text-align:right;">Grau de Ciència de Dades Aplicada (<i>Applied Data Science</i>)</p>
    <p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'Informàtica, Multimèdia i Telecomunicació</p>
</div>
<div style="width: 100%; clear: both;"></div>
<div style="width:100%;">&nbsp;</div>

# Extracció de dades: el cas de Twitter i la seva API

En aquest material veurem un cas particular d'extracció de dades d'una xarxa social: Twitter.

En primer lloc, necessitarem les claus de desenvolupador per poder accedir a l'API de Twitter. Si ja les heu sol·licitat, les podreu obtenir des del vostre compte. En cas contrari, visiteu https://developer.twitter.com/en/apply-for-access.html per obtenir informació i sol·licitar accés les claus.

La clau d'API de Twitter està, en realitat, format per quatre claus. Quan tingueu compte de desenvolupador, només cal accedir al menú **Apps** > **Create an app** i crear una *app* nova. Dins dels seus detalls trobareu **Keys and tokens**. Aquí es poden crear i regenerar les claus.

Introduïu les vostres claus API a la primera cel·la. Recordeu que no s'han de compartir per evitar l'ús fraudulent i també per evitar superar els límits de l'API.

In [ ]:
####input your credentials here
consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

Tot i que podríem fer crides directament a l'API, una de les millors maneres és aprofitar alguna llibreria addicional perquè gestioni alguns temes per nosaltres i que ens simplifiqui les coses. En aquest cas, farem servir `Tweepy` (http://docs.tweepy.org/en/v3.8.0/), encara que n'hi ha d'altres igualment vàlides com `Twython` (podeu fer servir la que preferiu, si coneixeu altres llibreries).

In [ ]:
import tweepy

Un exemple senzill (”hola món” de Tweepy) consisteix a descarregar els Tweets del nostre propi TL (*timeline*). Això inclou els comptes que seguiu. Si no en teniu cap, escriviu-ne un. Poseu un text genèric "Hola món", per exemple.

In [ ]:
# Preparem l'autenticació
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Preparem el mòdul api de Tweepy (que és el que ens ajudarà a fer les "preguntes" a la API)
api = tweepy.API(auth)

# I fem la primera crida: fixeu-vos que `api.home_timeline()` és una funció que retorna els tweets del nostre compte 
# timeline (de l'usuari de les claus)
public_tweets = api.home_timeline()

In [ ]:
# La crida retorna un conjunt de tweets que es poden iterar. Vegem els textos, per exemple.
for tweet in public_tweets:
    print(tweet.text)

## Check

Podeu comprovar com coincideixen amb el que veieu al client web de Twitter o la seva app mòbil.

# Recórrer i treure informació dels Tweets

Cada objecte que s'obté de `Tweepy` té diversos atributs. Abans hem vist que amb ".text" obtenim el text del tweet i, si busqueu a la documentació, en veureu molts més. Però el que ens serà més útil aquí serà utilitzar directament el JSON (JSON és un format estructurat de dades) en brut, accessible a partir de "._json".

Per exemple, imagineu que voleu veure els noms dels autors dels últims tweets del vostre TL:

In [ ]:
for tweet in public_tweets:
    print(tweet._json['user']['screen_name'])

Fixeu-vos que l'accés és com un diccionari, una forma que ja heu vist en el passat. Però... per què encara que el meu TL tingui centenars o milers de missatges només estic veient els darrers 20? Aquí hi ha una de les parts més rellevants de tot això.

## Límits

L'API de Twitter té límits, més laxos en alguns casos, més estrictes en altres. Teniu la referència a totes les funcions de l'API a https://developer.twitter.com/en/docs/api-reference-index, però per a l'explicació ens centrarem en dues: Search i Friends.

A Search (https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets), trobareu els detalls dels paràmetres per fer les trucades, juntament amb un exemple i una resposta d'exemple. A això ens ajudarà Tweepy, però hauríeu de fixar-vos una mica més amunt, a la part de "Resource information". Hi ha un apartat que posa "Rate Limited?" (Yes), cosa que significa que té màxims de preguntes. I més avall, "Requests/15-min window (app auth)" (450). Això el que significa és, bàsicament, que cada 15 minuts només podeu fer 450 preguntes de cerca. Sembla molt però no ho és; cada "pregunta" retorna com a molt 100 tweets (o 15 si no especifiquem res), de manera que obtenir dades sobre un hashtag o paraula popular pot consumir fàcilment les 450 crides. A més, la part gratuïta només dóna informació de cerca dels darrers 7 dies, així que cal tenir-ho en compte.

A Friends, que és una altra que podria ser interessant (per veure qui segueix un usuari concret), hi trobem un problema greu. Fixeu-vos que només permet 15! crides cada 15 minuts. És a dir, en una xarxa petita, de 1500 usuaris, estaríem unes 25 hores per treure tota la informació si no fallés res. La realitat és que, probablement, estiguem dia i mig o dos dies sencers. Tingueu-ho en compte si planegeu usar aquesta (interessant, això sí), relació.

Normalment gestionaríem això des de Python. Faríem una trucada a l'API i ens tornaria un valor (https://developer.twitter.com/en/docs/basics/response-codes ). "200" és l'indicador que tot està bé, però si torna un altre codi és que hi ha un error. "429", per exemple, és "Too many requests", indicador d'haver excedit el límit. Això ho processaríem com una excepció i, si hi entrés, esperaríem 15 minuts abans de tornar a trucar (o un temps prudencial). La sort és que Tweepy ho pot fer per nosaltres així que, per a l'objectiu d'aquest tutorial, no cal preocupar-se en excés (més enllà de tenir-ho en compte com a limitació d'extracció).

## Cerca

Farem un exemple de cerca amb Tweepy, que no tan sols gestiona els temps d'espera sinó també les extraccions més grans que la mida màxima de cerca - usant un invent anomenat cursor. Funciona així:

In [ ]:
# Fem una cerca de Tweets amb la paraula #Barcelona o #Madrid

# Es poden utilitzar operadors binaris (més detall a la documentació de la API)
for tweet in tweepy.Cursor(api.search,q=("#Barcelona OR #Madrid"),count=100,
                           tweet_mode= 'extended').items(200):
    print(tweet._json['full_text'])
    if 'retweeted_status' in tweet._json:
        print("Es un RT y su texto completo es: " + tweet._json['retweeted_status']['full_text'])

# Acabem de demanar els 200 tweets més recents, retornats en pàgines de 100 (que és el màxim)
# Amb les paraules #Barcelona o #Madrid
# tweet_mode = extended s'usa perquè des que Twitter va allargar els tweets (de 140 a 280 caràcters)
# s'emmagatzema el tweet de 280 caràcters en un altre atribut; el "text" normal estaria truncat.

Fixeu-vos en les diferències entre camps i en l'accés dins dels RT. Podeu jugar amb això i investigar.

## Problema: I l'emmagatzematge?

Fins aquest punt, cada vegada que busquem un conjunt de tweets a les instruccions anteriors, aquests es renoven o desapareixen. Així que interessa fer-ne una extracció i conservar-la. Aquí hi ha dues estratègies bàsiques:

- O bé es conserven els camps que siguin rellevants per a lexercici en un fitxer estructurat (per exemple, en un CSV que tingui per files els usuaris i les seves interaccions).

- O bé es fa servir una base de dades per a l'emmagatzematge. Aquí es recomana utilitzar MongoDB, que a més té una llibreria molt senzilla per a Python anomenada `pymongo`.

Totes dues opcions poden ser vàlides.

L'opció més senzilla és un CSV i es pot fer de múltiples maneres al gust del consumidor. Aquí un exemple, encara que normalment guardaríem més camps (usuari al qual es fa RT en cas de ser-ho, a qui respon, etc.). Noteu que si feu servir un CSV cal contemplar tots els casos i preparar tokens o comodins per a camps buits (e.g. si un Tweet no és un RT, posar un "*None*" en aquest camp). Cada fila ha de contenir el mateix nombre dítems, és important.

In [ ]:
# Guardem en un CSV fila per fila, ull que els caràcters separadors són importants
import csv

with open('tweet_list.csv', 'w') as csvfile:
            writer = csv.writer(csvfile, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            for tweet in tweepy.Cursor(api.search,q=("#Barcelona OR #Madrid"),count=100,
                           tweet_mode= 'extended').items(200):
                retweet = False
                if hasattr(tweet, 'retweeted_status'):
                    retweet = True
                writer.writerow([tweet._json['user']['id'], tweet._json['user']['screen_name'], tweet._json['full_text']])

In [ ]:
# I per una lectura ràpida i estructurada, fem servir pandas:
import pandas as pd

df = pd.read_csv('tweet_list.csv', header=None, sep=";")
df.head(10)

## Per MongoDB:

- Instal·lació: https://docs.mongodb.com/manual/administration/install-community/
- Inici: https://docs.mongodb.com/manual/tutorial/install-mongodb-on-windows/#run-mdb-edition-from-the-command-interpreter (cal iniciar el servei mongod cada vegada que es vulgui utilitzar).
- GUI (Compass): https://www.mongodb.com/download-center/compass

In [ ]:
# Exemple MongoDB

# Importem PyMongo
from pymongo import MongoClient

# Creem una connexió amb la BBDD
client = MongoClient()
           
# Utilitzem una base de dades anomenada test
db = client.test

# I cada Tweet s'emmagatzema en una col·lecció anomenada "tweets"
for tweet in tweepy.Cursor(api.search,q=("#Barcelona OR #Madrid"),count=100,
                           tweet_mode= 'extended').items(200):
    db.tweets.insert_one(tweet._json)

## Recuperació de MongoDB

Ara ja podríem treballar amb les dades que s'han recollit a la BBDD d'una manera ben directa: la nostra BBDD passaria a ser la font de tweets. Fixeu-vos en el seu ús base:

In [ ]:
import networkx as nx

G = nx.DiGraph()

for result in db.tweets.find():
            uid = result['user']['screen_name']
            G.add_node(uid) # si ja existeix, s'omet automàticament
            
            # Hi ha més casuístiques, però per exemple, amb el retweeted_status (és a dir, un usuari ha fet RT a un altre)
            if 'retweeted_status' in result:
                if G.has_edge(uid, result['retweeted_status']['user']['screen_name']):
                    G[uid][result['retweeted_status']['user']['screen_name']]['weight'] += 1.0
                else:
                    G.add_edge(uid, result['retweeted_status']['user']['screen_name'], weight = 1.0) 

Podeu fer servir Compass per veure el contingut de la BBDD.

# *Streaming*

Moltes vegades, però, a les plataformes en línia ens interessa "consumir" informació al moment. Si abans hem vist la cerca, ara veurem el streaming en temps real.

https://developer.twitter.com/en/docs/tweets/filter-realtime/overview

Una consideració important: si feu servir *streaming* per a una investigació té un problema de base. Twitter afirma que només ens proporciona un mostreig de, aproximadament, l'1% dels tweets que circulen en qualsevol moment i ho fa segons el seu algorisme de rellevància. Així que el que obteniu és un filtre sobre una mostra filtrada per la rellevància segons Twitter. Això podria ser bo però la realitat és que, com veureu, de vegades obtenim molt *spam* tuitero.

Com preparem un Stream amb Tweepy? Conceptualment és diferent del que hem vist fins ara, perquè en realitat el que farem és preparar un fil que deixarà el nostre programa "escoltant" del Stream de Twitter i cada vegada que arribi un tweet que coincideixi amb el nostre filtre saltarà a una rutina per tractar-lo (per exemple, per emmagatzemar-lo o escriure'l) i continuarà escoltant. Cal mantenir aquesta rutina al mínim per evitar que interfereixi amb el següent *tweet* (encara que es posi a una cua, si s'omple el *pipeline*, la connexió es trenca automàticament).

In [ ]:
# Per treballar amb json's
import json

# Llista amb les paraules
WORDS = ['#barcelona', '#madrid']

# Les claus
consumer_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
consumer_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
access_token_secret = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

# Crear la classe que realitzarà l'escolta
class StreamListener(tweepy.StreamListener):    
    # Aquesta és la classe que fa servir tweepy per accedir a la API de Streaming. 

    def on_connect(self):
        # Per connectar a la Streaming API
        print("You are now connected to the streaming API.")
 
    def on_error(self, status_code):
        # Si hi ha error no desconecta, però mostra l'error
        print('An Error has occured: ' + repr(status_code))
        return False
 
    def on_data(self, data):
        # Tractament al rebre "data" (un tweet)
        # Podríem, per exemple, connectar al mongoDB i desar el tweet
        try:
            
            # Codi d'exemple, però recordeu que els imports els hem indicat anteriorment
            '''client = MongoClient()
            db = client.testserver
            datajson = json.loads(data)
            db.tweets.insert_one(datajson)'''
            
            # Codificació de JSON a Python
            datajson = json.loads(data)
            
            # Obtenim el text
            text = datajson['text']
            fecha = datajson['created_at']

            # I printem un missatge indicant que s'ha capturat un Tweet            
            print("Tweet capturat a les "  + str(fecha) + " amb el text " + str(text))
           
        except Exception as e:
            print(e)

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Parem l'escolta (listener)
# Cal posar el 'wait_on_rate_limit=True' per gestionar les limitacions de la API de Twitter.
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
streamer = tweepy.Stream(auth=auth, listener=listener)

# Imprimir el que estem buscant
print("Tracking: " + str(WORDS))
streamer.filter(track=WORDS)

Els streams no tenen "fi". Cal aturar-los manualment i reiniciar el codi sencer si volem tornar a activar-lo i no tenir problemes.